In [1]:
from bert2head.model import BertCNN2HEAD, BertLinear2HEAD, BertCNN2HEAD_UIT
import torch



cuda


In [2]:
net = BertLinear2HEAD('vinai/phobert-base-v2')
net.load_state_dict(torch.load("models/linear/Best-bert-ever.pt"))


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.save(net.BertModel.state_dict(),"models/finetuneBertsmart.pt")

In [4]:
import torch
s = torch.ones(32,6)

In [13]:
import numpy as np

In [54]:
a = np.array([32,14,67,3,6]).reshape(-1,1)

np.random.randint(0,a)

array([[ 4],
       [ 4],
       [18],
       [ 2],
       [ 4]])

In [55]:
a=[torch.Tensor([4,3]),torch.Tensor([30,50])]
b=torch.cat(a).view(2,2)

In [56]:
b[b !=3] =1

In [57]:
b

tensor([[1., 3.],
        [1., 1.]])

In [40]:
c=torch.Tensor([0,0]).type(torch.IntTensor).reshape(-1,1)
c

tensor([[0],
        [0]], dtype=torch.int32)

In [44]:
b[torch.arange(2)[:,None],c]=1

In [45]:
b

tensor([[ 1.,  3.],
        [ 1., 50.]])

In [7]:
import torch

# Your sequence with 4 tokens
my_sequence = torch.Tensor([1,2,3,4,5,6,7,1,2,3,4,5,6,7])
print("Original sequence:", my_sequence)
# Choose a masking percentage based on the length of the sequence
masking_percentage = 0.15

# Determine the number of elements to select for masking
num_elements_to_mask = max(1, int(len(my_sequence) * masking_percentage))

# Randomly select elements for masking
masked_indices = torch.randint(0, len(my_sequence), size=(num_elements_to_mask,), dtype=torch.long)

# Create a mask for the selected indices
mask = torch.zeros(len(my_sequence))
mask[masked_indices] = 1
mask = mask.type(torch.IntTensor)
# Apply the mask to the sequence
#masked_sequence = [token if mask[i].item() == 0 else "[MASK]" for i, token in enumerate(my_sequence)]
my_sequence[mask]=-100

print("Masked sequence:", my_sequence)

Original sequence: tensor([1., 2., 3., 4., 5., 6., 7., 1., 2., 3., 4., 5., 6., 7.])
Masked sequence: tensor([-100., -100.,    3.,    4.,    5.,    6.,    7.,    1.,    2.,    3.,
           4.,    5.,    6.,    7.])


In [27]:
masked_indices = torch.randint(0, 10, size=(2,), dtype=torch.long)

In [8]:
import   copy

In [9]:

def random_label(input_ids: torch.Tensor,attention_mask: torch.Tensor, mlm_prob = 0.3):
      mlm_inputs = []
      labels =[]
      
      for input_id, att in zip(input_ids,attention_mask):
        mlm_input = input_id.clone()
        max_pos  = int(torch.sum(att))
        
        num_mask = int(max_pos * mlm_prob) 
              
        mask_pos = torch.randint(0, max_pos, size=(num_mask,), dtype=torch.int32)
        
        mask = torch.zeros(len(mlm_input))
        mask[mask_pos] = 1
        
        
        mask = mask.type(torch.bool)
        
        mlm_input[mask] = 64000
    
        label = copy.deepcopy(input_id)
        label[mlm_input != 64000]=-100
        
        lucky_mask = torch.randint(1,11,size = (num_mask,), dtype=torch.int32)
        for i,lucky in enumerate(lucky_mask):
          if int(lucky)%8==0: # 10%
            mlm_input[mask_pos[i]] = input_id[mask_pos[i]]
          elif int(lucky)%7==0: # 10%
            mlm_input[mask_pos[i]] = torch.randint(0,64000,size=(1,))[0]
        
        
        mlm_inputs.append(mlm_input.reshape(1,-1))
        labels.append(label.reshape(1,-1))

      mlm_inputs = torch.cat(mlm_inputs,dim=0)
      labels = torch.cat(labels,dim=0)
      return mlm_inputs, labels
      

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
model = AutoModelForMaskedLM.from_pretrained('vinai/phobert-base-v2')

In [55]:
inp = tokenizer('50.000.000',return_tensors='pt', padding=True,truncation=True, max_length=64)
inp

{'input_ids': tensor([[    0, 45382,     2]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [53]:
import torch

In [56]:
tokenizer.convert_ids_to_tokens(inp['input_ids'][0])

['<s>', '50.000.000', '</s>']

In [16]:
mlm_in, mlm_lb = random_label(inp['input_ids'],inp['attention_mask'])

NameError: name 'random_label' is not defined

In [14]:
out = model(input_ids=mlm_in,attention_mask=inp['attention_mask'])

In [17]:
def label_for_mlm(result, mlm_labels):
    y_pred=[]
    labels=[]
    for i in range(result.shape[0]):
        tmp_res = torch.zeros(result.shape[1])
        tmp_res[mlm_labels[i]!=-100] =1
        
        for j in range(result.shape[1]):
            if tmp_res[j]==1:
                lb = mlm_labels[i,j]
                pred = out['logits'][i][j]
                y_pred.append(pred.reshape(1,-1))
                labels.append(lb)

    labels= torch.Tensor(labels).type(torch.LongTensor)
    y_pred = torch.cat(y_pred,dim=0)  
    
    return y_pred, labels

In [18]:
y_pred,labels = label_for_mlm(out['logits'],mlm_lb)

In [19]:
import torch.nn.functional as F
loss2 = F.cross_entropy(y_pred, labels)

In [20]:
loss2

tensor(3.8681, grad_fn=<NllLossBackward0>)

In [42]:
import pandas as pd
df = pd.read_csv('dataset/MLMdata.csv')

In [43]:
df

,Unnamed: 0,text
0,0,vòng kết đến .
1,1,khí lớp_học vui_vẻ thoải_mái .
2,2,hướng học kinh_tế vi_mô photo .
3,3,in sẵn banner luôn .
4,4,inbox mua het nhe ban .
...,...,...
30030,30030,hào thành_viên cụm truyền_thông kiện hội sinh_...
30031,30031,giảng_viên giảng ví_dụ minh_họa giúp sinh_viên...
30032,30032,hai vai_trò thầy wzjwz môn wzjwz kỹ_thuật ngôn...
30033,30033,chương_trình tân_sinh_viên làn_sóng đột_phá nă...


In [44]:
test = pd.read_csv('dataset/Test/test2.csv')

In [45]:
test

,Unnamed: 0,text,label_x,label_y
0,0,chúc các bạn luôn thành_công và thành_đạt trên...,1,7
1,1,tuyệt_vời quá đi .,1,3
2,2,theo các chuyên_gia của đại_học kinh_tế quốc d...,0,7
3,3,chúc con luôn đạt nhiều ngoạn_mục trên con đườ...,1,6
4,4,trường tâm_lý .,1,4
...,...,...,...,...
2775,2780,giảng_viên chỉ cho đồ_án và giải_đáp các yêu_c...,2,2
2776,2781,cô nên dạy nội_dung khó hơn cho lớp sau .,2,2
2777,2782,em chưa ạ .,2,3
2778,2783,thấy cao lắm hích hích .,0,3


In [46]:
test = set(" ".join(x.split()) for x in test['text'].values)

In [47]:
arr = []
for t in df['text'].values:
    text = " ".join(t.split())
    if text not in test:
        arr.append(text)

In [48]:
df2 = pd.DataFrame({'text':arr})

In [49]:
df2

,text
0,vòng kết đến .
1,khí lớp_học vui_vẻ thoải_mái .
2,hướng học kinh_tế vi_mô photo .
3,in sẵn banner luôn .
4,inbox mua het nhe ban .
...,...
30030,hào thành_viên cụm truyền_thông kiện hội sinh_...
30031,giảng_viên giảng ví_dụ minh_họa giúp sinh_viên...
30032,hai vai_trò thầy wzjwz môn wzjwz kỹ_thuật ngôn...
30033,chương_trình tân_sinh_viên làn_sóng đột_phá nă...


In [50]:
df2.to_csv('dataset/MLMdata.csv')

In [53]:
df2.shape


(30035, 1)